In [35]:
from ff3 import FF3Cipher
import string
import re
import secrets
from itertools import islice

In [36]:
def batched(iterable, n, *, strict=False):
    # batched('ABCDEFG', 3) → ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    iterator = iter(iterable)
    while batch := tuple(islice(iterator, n)):
        if strict and len(batch) != n:
            raise ValueError('batched(): incomplete batch')
        yield batch

In [16]:
key = secrets.token_hex(16)
tweak = secrets.token_hex(7)

In [104]:
def _modify_chunk(chunk, alpha_ind=False):
    if alpha_ind and len(chunk) < 7:
        return chunk + (7 - len(chunk))*("A",)
    elif len(chunk) < 7:
        return chunk + (7 - len(chunk))*("0",)
        
    else:
        return chunk

In [107]:
def process_chunks(c_input, c_cipher, alpha_ind=False):
    return ''.join([
        c_cipher.encrypt(_modify_chunk(chunk, alpha_ind))
        for chunk in batched(c_input, 32)
    ])        

In [78]:
def process_chunks_1(letters, c_letters):
    return ''.join([
        c_letters.encrypt(_modify_chunk(chunk))
        for chunk in batched(letters, 32)
    ])        

In [108]:
def encrypt_string(plain_text, key, tweak):
    
    c_numbers =  FF3Cipher(key, tweak)
    c_letters = FF3Cipher.withCustomAlphabet(key, tweak, string.ascii_letters)
    pattern_letters = re.compile(r'[^a-zA-Z]')
    pattern_nums = re.compile(r'\D+')
    letters = pattern_letters.sub('', plain_text)
    nums = pattern_nums.sub('', plain_text)
    
    if len(nums) < 7:
        nums =  nums + (7-len(nums))*"0"
        
    ciphertext = process_chunks(letters, c_letters, alpha_ind=True)
    ciphernum = process_chunks(nums, c_numbers)
    #ciphernum = c_numbers.encrypt(nums)
    encrypted_text = []
    letter_idx, digit_idx = 0, 0
    for char in plain_text:
        if char in letters:
            encrypted_text.append(ciphertext[letter_idx])
            letter_idx += 1
        elif char in nums:
            encrypted_text.append(ciphernum[digit_idx])
            digit_idx += 1
        else:
            encrypted_text.append(char)
        
    return ''.join(encrypted_text)

In [111]:
encrypt_string("12345999999, # & this is nice"*55, key, tweak)

'66300499660, # & YXLu FY KGaj50284003653, # & wgQJ kw CGlJ91488453126, # & lbNa PW ebxY16728693610, # & olfF MT NHZW46862852605, # & ueLO dO hAxf01870734512, # & NYnf Tl PEXq34169508230, # & Hjja Yu johf36519523449, # & ZaqE nE sMlU80202633968, # & oRQU ft fuSw29628219632, # & rVjK my HbcR83787960850, # & AYuy kf SJKK97925445435, # & TRiA NM igFj23498980942, # & iVRH cH ayju39761966033, # & jRqe eG vrKr67117606620, # & puxb zg cBfE41015644813, # & czrR aB ImGm82841768459, # & YXLu FY KGaj26808806973, # & wgQJ kw CGlJ51759148962, # & lbNa PW ebxY13010831196, # & olfF MT NHZW67227548775, # & ueLO dO hAxf31271078318, # & NYnf Tl PEXq41293819511, # & Hjja Yu johf70606808474, # & ZaqE nE sMlU84818829280, # & oRQU ft fuSw66689116468, # & rVjK my HbcR25728082962, # & AYuy kf SJKK39263164468, # & TRiA NM igFj70580278367, # & iVRH cH ayju95429091370, # & jRqe eG vrKr97104344410, # & puxb zg cBfE58293952751, # & czrR aB ImGm66300499660, # & YXLu FY KGaj50284003653, # & wgQJ kw CGlJ91488453126, 